In [1]:
import nest_asyncio
nest_asyncio.apply()

from ib_insync import util
util.startLoop()

In [72]:
# Simulate a big qualifyAsync
from ib_insync import IB, Option, Contract, util
import pandas as pd
import yaml
import asyncio
import random
import logging
from tqdm import tqdm

MARKET = 'snp'
root_path = "C:/Users/User/Documents/Business/Projects/asyncib/"
data_path = root_path+'data/'+MARKET.lower()+'/'

# get the connection IDs
with open(root_path+'var.yml') as f:
    data=yaml.safe_load(f)
    
HOST = data["COMMON"]["HOST"]
PORT = data[MARKET.upper()]["PORT"]
CID = data["COMMON"]["CID"]

# make the contract chains
df_chains = pd.read_pickle(data_path+'df_chains.pkl')

puts = [Option(symbol=s, lastTradeDateOrContractMonth=e, strike=k, right='P', exchange=x) 
                for s, e, k, x in 
                zip(df_chains.symbol, df_chains.expiry, df_chains.strike, ['NSE' 
                    if MARKET.upper() == 'NSE' else 'SMART']*len(df_chains))]

calls = [Option(symbol=s, lastTradeDateOrContractMonth=e, strike=k, right='C', exchange=x) 
                for s, e, k, x in 
                zip(df_chains.symbol, df_chains.expiry, df_chains.strike, ['NSE' 
                    if MARKET.upper() == 'NSE' else 'SMART']*len(df_chains))]

raw_cs = puts+calls

In [122]:
contracts = random.sample(raw_cs, 5) # sample list of contracts
util.logToConsole(level=logging.FATAL) # Suppress ib_insync errors

In [132]:
from pprint import pprint
# create an async task
async def qual(contracts):
    blks = 100
    tasks = list((ib.qualifyContractsAsync(c) for c in contracts))
    done, pending = await asyncio.wait(tasks)
    
    pprint(done)
    pprint(pending)
    
    return tasks

In [136]:
%%time
with IB().connect(HOST, PORT, CID) as ib:
    result = ib.run(qual(contracts))

{<Task finished name='Task-180' coro=<IB.qualifyContractsAsync() done, defined at C:\Users\User\AppData\Local\Programs\Python\Python38-32\lib\site-packages\ib_insync\ib.py:1627> result=[]>,
 <Task finished name='Task-179' coro=<IB.qualifyContractsAsync() done, defined at C:\Users\User\AppData\Local\Programs\Python\Python38-32\lib\site-packages\ib_insync\ib.py:1627> result=[Option(conId=...ngClass='XEO')]>,
 <Task finished name='Task-178' coro=<IB.qualifyContractsAsync() done, defined at C:\Users\User\AppData\Local\Programs\Python\Python38-32\lib\site-packages\ib_insync\ib.py:1627> result=[]>,
 <Task finished name='Task-182' coro=<IB.qualifyContractsAsync() done, defined at C:\Users\User\AppData\Local\Programs\Python\Python38-32\lib\site-packages\ib_insync\ib.py:1627> result=[Option(conId=...ngClass='CVX')]>,
 <Task finished name='Task-181' coro=<IB.qualifyContractsAsync() done, defined at C:\Users\User\AppData\Local\Programs\Python\Python38-32\lib\site-packages\ib_insync\ib.py:1627> re

In [137]:
result

[<coroutine object IB.qualifyContractsAsync at 0x0E2EA7A8>,
 <coroutine object IB.qualifyContractsAsync at 0x0E379928>,
 <coroutine object IB.qualifyContractsAsync at 0x01863D28>,
 <coroutine object IB.qualifyContractsAsync at 0x01863BE8>,
 <coroutine object IB.qualifyContractsAsync at 0x0B44B428>]